In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Geometric Properties of Curves

To move beyond visual evaluations of a curve we require means of quantification. Here we describe the essential mathematical quantities that capture the geometric features of curves most often used in design.

These include quantities such as ***length***, ***curvature***, and ***Frenet frame*** (which includes the ***unit tangent*** and ***normal*** vectors).

All of These are typically expressed using calculus and implemented using numerical methods. Here we will limit ourselves to a discrete setting where the curve is thought of as a sampling of points, sometimes called ***discrete approximation***.

An easy way to understand how this works is to look at how curve length is calculated.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P10.jpg" style="width: 200px; display: inline;">

### Curve Length

The simplest way of calculating the length of a curve is to sample the curve to produce a dense collection of points, and then sum up the distance between each pair of points.

In code, this would be expressed as such:

In [ ]:
"""
Curve Length
"""
def appx_length(self): 
    length = 0
    for ival in Interval() // resolution:
        length += self.eval(ival.a).distance(self.eval(ival.b))
    return length

The more samples we take, the more accurate our length calculation. 

Take, for example, a series of length calculations for a parabolic curve, each with an increasing number of samples. The results are shown in the table bleow, which demonstrates two properties of this discrete approach to calculating curve length:
* the additional number of samples required to achieve an additional decimal point of precision increases exponentially
* the calculated length increases with additional samples, converging from below on the actual length of the Curve.

<table style="width:600px">
    <tr>
        <th colspan="2" style="text-align:left">*Curve Length Accuracy*</th>
    </tr>
    <tr>
        <th style="text-align:left">*Number of Divisions*</th>
        <th style="text-align:left">*Calculated Length*</th>
    </tr>
    <tr>
        <td>2</td>
        <td><span style="color:blue">4.57649122254</span></td>
    </tr>
    <tr>
        <td>4</td>
        <td>4.<span style="color:blue">62672348734</span></td>
    </tr>
    <tr>
        <td>16</td>
        <td>4.6<span style="color:blue">4552053219</span></td>
    </tr>
    <tr>
        <td>32</td>
        <td>4.64<span style="color:blue">646795934</span></td>
    </tr>
    <tr>
        <td>128</td>
        <td>4.646<span style="color:blue">76402483</span></td>
    </tr>
    <tr>
        <td>512</td>
        <td>4.6467<span style="color:blue">8252883</span></td>
    </tr>
    <tr>
        <td>1084</td>
        <td>4.64678<span style="color:blue">345403</span></td>
    </tr>
</table>

## Calculating Properties by Nearest Neighbors Approximation

To illustrate and compute the remainder of the geometric properties, we build upon this basic logic of extracting information on neighboring points, usually in groups of three points at a time.

The properties of ***curvature***, ***unit tangent***, ***normal vector***, and ***Frenet frame*** all describe certain local qualities of a curve near a particular point. The discrete approximation of these local properties requires additional information besides the point itself; the addition of two of its ***nearest neighbors***, points that are just ahead and just behind our point of interest.

To ensure that the neighboring points are close, we set the step-size $\Delta t$ to be a fraction of `Curve.tol`.

In [ ]:
"""
Calculation of Nearest Neighbors 
Returns a Point on the Curve and two neighboring Points on either side at a distance related to 
the tolerance of the Curve 
"""
def nearest_neighbors(crv,t):
    pt = crv.eval(t)
    pt_plus = crv.eval(t + crv.tol_nudge)
    pt_minus = crv.eval(t - crv.tol_nudge)
    return pt, Vec(pt_plus,pt), Vec(pt,pt_minus)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P16.jpg" style="width: 200px; display: inline;">

### Tangent Vector

The ***tangent vector*** at a curve point measures the rate of change of the curve.

Geometrically, this is a vector that just touches the curve at the curve point. In a discrete setting, the vectors that connect the point to its nearest neighbors are approximately vectors tangent to the curve. 

If all we want is the direction of the tangent, then normalizing any one of these vectors gives an approximation for the ***unit tangent vector***, denoted as $\vec{T}$

In [ ]:
"""
Unit Tangent Approximation
"""
pt_t, vec_plus, vec_minus = nearest_neighbors(crv,t)
unit_tangent = vec_plus.normalized() 

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P07.jpg" style="width: 200px; display: inline;">

### Normal Vector

Unlike a tangent vector, ***there are many possible normal vectors*** to a curve. In fact, when the curve is in three dimensions, there is *an entire plane of eligible vectors*.

To define a *special normal vector* using a discrete approach, take three points – the point and its nearest neighbors – which determine a plane and a circle. These are called the ***osculating plane*** and ***osculating circle*** at the curve point. 

The vector that *connects the curve point to the center of the circle* is perpendicular to the tangent vector, and is thus normal to the curve. 

The unit vector in this normal direction is called the ***principal normal vector***, denoted as $\vec{N}$

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P09.jpg" style="width: 200px; display: inline;">

### Curvature

The curvature at a curve point gives a numerical measurement of the turning of a curve, and is defined to be the reciprocal value $x = 1/R$, where $R$ is the radius of the osculating circle at the curve point.

The smaller the circle, the tighter the turning and in turn, the bigger the curvature becomes.

Calculating curvature using nearest neighbor vectors is a built-in method of the Curve class and amounts to finding the center and radius of the approximate osculating circle.

In [1]:
"""
Curvature From Vectors Method in Curve Class
Returns curvature and circle determined by point and nearest neighbors on either side
"""
def _curvature_from_vecs(pt, vec_pos, vec_neg, calc_circles=False):
    pt_plus = pt + vec_pos
    pt_minus = pt + vec_neg
    
    v1 = vec_pos
    v2 = vec_neg
    v3 = Vec(vec_pos - vec_neg)
    
    xl = v1.cross(v3).length
    if xl == 0 : return 0,Ray(pt,vec_pos)
    
    rad_osc = 0.5*v1.length*v2.length*v3.length/xl
    if not calc_circles: return 1/rad_osc
   
    denom = 2*xl*xl
    a1 = v3.length*v3.length*v1.dot(v2)/denom
    a2 = v2.length*v2.length*v1.dot(v3)/denom
    a3 = v1.length*v1.length*(-v2.dot(v3))/denom
    center_osc = pt*a1 + pt_plus*a2 + pt_minus*a3
    
    pln_out = Plane(center_osc, v1.cross(v2))
    circ_out = Circle(pln_out,rad_osc)
    return (1/rad_osc, circ_out)

In [ ]:
"""
Curvature 
Evaluates curvature at at given t-value
"""        
def deval_crv(self,t):
    pt, vec_pos, vec_neg = self._nudged(t)

    # nudge vectors to avoid zero curvature at endpoints
    if (t-self.tol_nudge <= self.domain.a):
        nhood = self._nudged(self.tol_nudge)
        vec_pos = nhood[1]
        vec_neg = nhood[2]
    if (t+self.tol_nudge >= self.domain.b):
        nhood = self._nudged(self.domain.b-self.tol_nudge)
        vec_pos = nhood[1]
        vec_neg = nhood[2]

    return Curve._curvature_from_vecs(pt,vec_pos,vec_neg)

In [ ]:
"""
Curvature
Evaluates curvature at normalized t-value
"""         
def eval_crv(self,t):
    return self.deval_crv(Interval.remap(t,Interval(),self.domain))  

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P18.jpg" style="width: 800px; display: inline;">

### Frenet Frame

The Frenet frame is linked to the shape of the curve, and describes how it turns and twists in space. It is composed of a set of three orthonormal vectors. 

We have already seen two of these vectors: the ***unit tangent vector $\vec{T}$*** and the ***principal unit normal $\vec{N}$***. 

The third vector in the set, called the ***binormal vector***, is a simple product of  these first two. Denoted by $\vec{B}$, the binormal vector results from taking the cross product $\vec{T} \times \vec{N}$ .

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P11.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Curve CS
Evaluates a CS aligned with Frenet Frame at given t-value
"""  
def deval_cs(self,t):
    pt, vec_pos, vec_neg = self._nudged(t)

    # nudge vectors to avoid zero curvature at endpoints
    if (t-self.tol_nudge <= self.domain.a):
        nhood = self._nudged(self.tol_nudge)
        vec_pos = nhood[1]
        vec_neg = nhood[2]
    if (t+self.tol_nudge >= self.domain.b):
        nhood = self._nudged(self.domain.b-self.tol_nudge)
        vec_pos = nhood[1]
        vec_neg = nhood[2]
    vec_T = self.tangent(t)
    k, circ = Curve._curvature_from_vecs(pt,vec_pos,vec_neg,calc_circles=True)
    center_osc = circ.plane.origin
    vec_N = Vec(center_osc-pt).normalized()
    vec_B = vec_T.cross(vec_N)
    return CS(pt, vec_N, vec_B)

In [ ]:
"""
Curve CS
Evaluates a CS aligned with Frenet Frame at normalized t-value
"""          
def eval_cs(self,t):
    return self.deval_cs(Interval.remap(t,Interval(),self.domain))

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P12.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Propagate Polygons Along Curve
Generates a collection of polygons using the built-in CS evaluated along curve
"""
pgons = []
for t in Interval().divide(pgon_count):
    cs = crv.eval_cs(t)
    rad_out = Interval(r1,r0*0.25).eval(t**(1/pow))
    rad_in = Interval(r0,r1*0.25).eval(t**(1/pow))
    pgon = PGon.doughnut(cs,Interval(rad_out,rad_in),res=edge_count)
    pgons.append(pgon)

"""
Variable Pipe Around Curve
Meshes Resulting Collection of Polygons
"""
for n in range(len(pgons)-1):
    pgon_a = pgons[n]
    pgon_b = pgons[n+1]
    off = len(pgon_a.pts)
    
    msh = Mesh()
    msh.append(pgon_a.pts)
    msh.append(pgon_b.pts)
    
    for e in range(off):
        if e != (off/2)-1:
            msh.add_face(e,e+1,e+off+1)
            msh.add_face(e+off+1,e+off,e)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P17.jpg" style="width: 800px; display: inline;">